<a href="https://colab.research.google.com/github/relaypy/MAIS202_Project/blob/main/Recipee_recomend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Start by importing the dataset</h1>
<p>as you can see I ve imported recipee dataset from kaggle with over 2 million recipee, ofc it would be unreaseanable to start directly trying to build our model directly so we ll have to do some data processing before that </p>


In [4]:
import pandas as pd
import os
import kagglehub


# Download latest version
path = kagglehub.dataset_download("paultimothymooney/recipenlg")
final = path+"/RecipeNLG_dataset.csv"
dataset = pd.read_csv(final)
dataset.head()


100%|██████████| 643M/643M [00:07<00:00, 89.2MB/s]

Extracting files...


,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


<h1><b>Data processing</b></h1>
<p> Here in the dataprocessing section i decided to start first by shortening my the number of features from 2million to 5000 , the main reason why i did this is because treating all those 2M points would be very caustly while training due to computing power limitations, further more I had to creat another table where I drop a bunch of useless columns like the title the link or the source since in this case we re only interested in the ingredients and the id of the recipee.</p>
<p>But we re not done yet , the main challenge is still to come, At this point I had to question my self how am I supposed to create a trainable dataset since the concern that I have right now is that each cell in my "NER" columns contain a string of ingredients so I had to figure out some way to represent the array into scalar point. With the magic of googling I have discovered the concept of hot-encoding , basically the idea is to take all the possible worlds(in our case all the possible ingredients), and add a collumn to our dataset for each word and for every feature represent the presence of the word by 1 or the absence of the word 0. In my Case I went and selected just the most communly used ingredients (an ingredients that has been used for atleast 50) , with the power of for loop I was able to do this encoding and add these new columns to my shortened dataset while dropping the initial "NER" that became useless



In [105]:
dataset_short = dataset.iloc[:5000, :]
dataset_short.drop(['ingredients','directions','link','source'],axis=1,inplace =True)
dataset_short.rename(columns={'Unnamed: 0':'id'},inplace=True)
dataset_short.head()

import json

all_ingr = []
freq_of_ingr=[]
for i in range(5000):
  for j in json.loads(dataset_short.iloc[i,2]) :
    if not j in all_ingr:
      all_ingr.append(j)
      freq_of_ingr.append(1)
    else:
      for i in range(len(all_ingr)):
        if all_ingr[i] == j:
          freq_of_ingr[i]+=1




print(len(all_ingr))
print(len(freq_of_ingr))
final_list = []
for i in range(len(all_ingr)):
  if freq_of_ingr[i]>=50:
    final_list.append(all_ingr[i])
print(len(final_list))



for i in final_list:
  dataset_short[i] = 0


for i in range(5000):
  for j in json.loads(dataset_short.iloc[i,2]) :
    if j in final_list:
      dataset_short.at[i,j] =1

dataset_short.drop(["title","NER"],axis=1 , inplace=True)
dataset_short.head()



<ipython-input-105-bbdb11375d54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_short.drop(['ingredients','directions','link','source'],axis=1,inplace =True)
<ipython-input-105-bbdb11375d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_short.rename(columns={'Unnamed: 0':'id'},inplace=True)


2781
2781
113


<ipython-input-105-bbdb11375d54>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_short[i] = 0
<ipython-input-105-bbdb11375d54>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_short[i] = 0
<ipython-input-105-bbdb11375d54>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,id,brown sugar,milk,vanilla,nuts,butter,chicken breasts,cream of mushroom soup,sour cream,cream cheese,...,almonds,chili powder,cottage cheese,white sugar,pumpkin,paprika,bread crumbs,vanilla extract,Crisco,red pepper
0,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1><b>The Model</b></h1>
<p>For the model AKA the fun part and the also partially the reason why I am doing this project I've decided to go for Simple feedforward or MLP model that uses softmax for predicting the the index(the id) based on the features available,We can quite arguably say that some other model would be better in this case , for instance Maybe we could have used a Word2Vec encoding technique and then used an RNN to have in which case we would have fed into our data set maybe the description of the recipee instead of just ingredient,in which case I would agree that it would be better.But I really wanted to go for he simple MLP model for multitude of reasons,mainly personnal , one its because its probably my first time using pytorch and I thought it would a better idea to not rush things through and start of bye the basics , second this is actually not the end of the project but rather a first attempt to creat this recommendation engine and i still planing to improve this model to include much more than just ingredients but might also include a Price range , or maybe some diatery restriction.</p>


In [111]:
import numpy as np

X_train = np.array(dataset_short.iloc[:,1:])
Y_train = np.array(dataset_short.iloc[:,0])


import torch.nn as nn
import torch


class MyMLP(nn.Module):
  def __init__(self, D,F):
    super(MyMLP, self).__init__()
    self.l1 = nn.Linear(D, 1000)
    self.l2 = nn.Linear(1000, 1000)
    self.l3 = nn.Linear(1000, F)

  def forward(self, x):
    a1 = self.l1(x)
    z1 = torch.relu(a1)

    a2 = self.l2(z1)
    z2= torch.relu(a2)

    y_hat = self.l3(z2)


    return y_hat



n_epochs = 50
batch_size = 100
learning_rate = 0.01

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)

model = MyMLP(X_train.shape[1],Y_train.shape[0])
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

losses = []

for _ in range(n_epochs):
  total = 0
  epoch_loss = 0
  correct = 0
  for x_batch, y_batch in train_loader:

    optimizer.zero_grad()

    y_hat = model(x_batch)

    l = loss(y_hat, y_batch)
    epoch_loss += l.item()

    l.backward()

    optimizer.step()

    _, predicted = torch.max(y_hat, 1)  # Get the predicted class (index with max logit)
    total += y_batch.size(0)
    correct += (predicted == y_batch).sum().item()

  epoch_loss /= len(train_loader)
  accuracy = 100 * correct / total
  losses.append(epoch_loss)

In [112]:
print(accuracy)
print(losses)

82.26
[8.625904922485352, 8.539828758239747, 8.14076473236084, 6.735755405426025, 4.483714690208435, 2.3487516117095946, 1.3566513514518739, 0.9561210322380066, 0.7843561917543411, 0.6936542177200318, 0.634843944311142, 0.6048440682888031, 0.5906311571598053, 0.5605863118171692, 0.5606120747327804, 0.5364497995376587, 0.5363176745176316, 0.5246195596456528, 0.5194313329458237, 0.5144324117898941, 0.5089164024591446, 0.5015239331126213, 0.5003154307603837, 0.49838062733411786, 0.4930371290445328, 0.48814320623874663, 0.489189425110817, 0.4863110816478729, 0.4839412701129913, 0.48664516627788545, 0.4953589367866516, 0.50481201171875, 0.5293185138702392, 0.5904732716083526, 0.6504425472021103, 0.8663298428058624, 1.2261076521873475, 1.2240039777755738, 1.0219796776771546, 0.8026948261260987, 0.6424811053276062, 0.5701900342106819, 0.501764754652977, 0.47025167405605317, 0.4592508065700531, 0.4572455859184265, 0.45095584720373155, 0.4476429840922356, 0.4495242699980736, 0.44614422082901]


<h1><b>Evaluation and Testing</b></h1>
<p>A less complicated but no as fun as building the model, u can see above an accuracy metric but this doesn't reflect very well on the actual performence and I think honestly that the most effective way to test this model or especially any recommendation engine is through user satisfaction or user rating and comments that's  why i decided to do a test with an array that i constructed accoding to some ingredients that I have on my fridge right now, and decided to pich the three most recommended recipee by the model , and i got a smewhat satisfactory answer.</p>




In [119]:
model.eval()
test1 = ['butter','egg','chicken breasts','potatoes','soy sauce']
test2 = [0]*len(final_list)

for i in range(len(final_list)):
  if final_list[i] in test1 :
    test2[i] =1


with torch.no_grad():
    test3 = torch.tensor(np.array([test2]), dtype=torch.float32)
    y_hat = model(test3)
    topk_values, topk_indices = torch.topk(y_hat, k=3, dim=1)
    _, predicted = torch.max(y_hat, 1)
    top=[]
    for i in range(3):
      top.append(topk_indices[0,i].item())

for i in top:
  print(dataset.iloc[i,1]+":"+dataset.iloc[i,6])


Chicken Tarragon En Croutte:["butter", "lemon juice", "tarragon", "garlic salt", "pepper", "crescent rolls", "chicken breasts", "flour", "egg"]
Potato Cakes:["potatoes", "onion", "egg", "flour"]
Lefse(Norwegian)  :["potatoes", "butter", "milk", "flour"]
